# 📊 Optimisation de Production - Programmation Linéaire

**Projet :** Optimisation de la production de canapés avec contraintes de ressources  
**Auteur :** Emmanuel Paguiel BOUENDO  
**Date :** 2024-2025  
**Contexte :** Master Économiste de l'Entreprise - Université de Tours

---

## 🎯 Objectif du projet

Ce notebook présente une **analyse complète d'optimisation de production** utilisant la programmation linéaire. L'entreprise doit maximiser son bénéfice sous contraintes de temps, matières premières et capacités de marché.

### 📌 Compétences démontrées
- Modélisation mathématique de problèmes économiques
- Programmation linéaire et optimisation
- Analyse de sensibilité
- Visualisation de données
- Python scientifique (NumPy, SciPy, Matplotlib)

In [ ]:
# Imports des bibliothèques nécessaires
import numpy as np
import scipy.optimize as so
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import pandas as pd
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings('ignore')

# Configuration pour de meilleurs graphiques
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['legend.fontsize'] = 10

print("✅ Bibliothèques importées avec succès")

---

## 📋 Introduction - Contexte du problème

Une entreprise de fabrication de meubles produit des canapés de différents modèles. Elle commence par analyser le marché avec deux modèles **A** et **B**.

### 💰 Données économiques

| Modèle | Bénéfice unitaire | Temps de production | Prix de vente |
|--------|-------------------|---------------------|---------------|
| **A**  | 300 €             | 2 heures           | 800 € |
| **B**  | 200 €             | 1 heure            | 500 € |

### ⚙️ Contraintes de production

1. **Temps de production** : L'usine dispose de **200 heures/mois**
2. **Capacité de marché** :
   - Maximum **100 canapés de type A** par mois
   - Maximum **100 canapés de type B** par mois
   - Maximum **150 canapés au total** par mois
3. **Contraintes de matières premières** (voir partie B)

### 🎯 Question centrale

**Combien de canapés de chaque type l'entreprise doit-elle produire pour maximiser son bénéfice ?**

---

## 📐 Partie 1 : Modélisation mathématique

### 🔢 Formulation du problème de programmation linéaire

**Variables de décision :**
- $x_1$ : nombre de canapés de modèle A produits
- $x_2$ : nombre de canapés de modèle B produits

**Fonction objectif (à maximiser) :**
$$\max Z = 300x_1 + 200x_2$$

**Contraintes :**
$$
\begin{aligned}
& 0 \leq x_1 \leq 100 & \text{(capacité marché A)} \\
& 0 \leq x_2 \leq 100 & \text{(capacité marché B)} \\
& 2x_1 + x_2 \leq 200 & \text{(temps de production)} \\
& x_1 + x_2 \leq 150 & \text{(capacité totale)}
\end{aligned}
$$

In [ ]:
# Définition des paramètres du problème
class ProblemeOptimisation:
    """Classe pour encapsuler le problème d'optimisation"""
    
    def __init__(self):
        # Coefficients de la fonction objectif (bénéfices)
        self.c = [-300, -200]  # Négatif car scipy minimise par défaut
        
        # Contraintes d'inégalité (A_ub @ x <= b_ub)
        self.A_ub = np.array([
            [2, 1],   # Temps de production
            [1, 1]    # Capacité totale
        ])
        self.b_ub = np.array([200, 150])
        
        # Bornes des variables
        self.bounds = [(0, 100), (0, 100)]  # x1 et x2 entre 0 et 100
        
    def fonction_objectif(self, x):
        """Calcule le bénéfice pour un plan de production donné"""
        return 300 * x[0] + 200 * x[1]
    
    def afficher_parametres(self):
        """Affiche les paramètres du problème"""
        print("📊 Paramètres du problème d'optimisation")
        print("=" * 50)
        print(f"Bénéfice modèle A : {-self.c[0]} €")
        print(f"Bénéfice modèle B : {-self.c[1]} €")
        print(f"\nContrainte temps : 2x₁ + x₂ ≤ {self.b_ub[0]}")
        print(f"Contrainte capacité : x₁ + x₂ ≤ {self.b_ub[1]}")
        print(f"\nBornes : 0 ≤ x₁ ≤ {self.bounds[0][1]}, 0 ≤ x₂ ≤ {self.bounds[1][1]}")

# Création de l'instance du problème
probleme = ProblemeOptimisation()
probleme.afficher_parametres()

---

## 🔍 Partie 2 : Résolution numérique

In [ ]:
# Résolution avec la méthode du simplexe (scipy.optimize.linprog)
def resoudre_probleme(probleme):
    """Résout le problème de programmation linéaire"""
    
    result = so.linprog(
        c=probleme.c,
        A_ub=probleme.A_ub,
        b_ub=probleme.b_ub,
        bounds=probleme.bounds,
        method='highs'  # Algorithme moderne et efficace
    )
    
    if result.success:
        x1_opt, x2_opt = result.x
        benefice_max = -result.fun  # Repassage en maximisation
        
        print("\n" + "=" * 60)
        print("🎯 SOLUTION OPTIMALE TROUVÉE")
        print("=" * 60)
        print(f"\n📦 Production optimale :")
        print(f"   • Canapés modèle A : {x1_opt:.0f} unités")
        print(f"   • Canapés modèle B : {x2_opt:.0f} unités")
        print(f"   • Total canapés    : {x1_opt + x2_opt:.0f} unités")
        print(f"\n💰 Bénéfice maximal : {benefice_max:,.0f} €")
        print(f"\n⏱️  Utilisation du temps : {2*x1_opt + x2_opt:.0f} h / 200 h ({100*(2*x1_opt + x2_opt)/200:.1f}%)")
        print(f"📊 Taux d'utilisation capacité : {100*(x1_opt + x2_opt)/150:.1f}%")
        print("=" * 60)
        
        return x1_opt, x2_opt, benefice_max
    else:
        print("❌ Échec de l'optimisation")
        return None, None, None

# Résolution
x1_opt, x2_opt, max_benefice = resoudre_probleme(probleme)

---

## 📈 Partie 3 : Visualisation graphique de la solution

In [ ]:
def visualiser_solution(x1_opt, x2_opt, max_benefice):
    """Crée une visualisation complète de la région réalisable et de la solution"""
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Génération des points
    x1 = np.linspace(0, 110, 1000)
    
    # Droites de contraintes
    x2_temps = 200 - 2*x1        # 2x1 + x2 = 200
    x2_capacite = 150 - x1       # x1 + x2 = 150
    x2_marche_B = np.full_like(x1, 100)  # x2 = 100
    
    # Tracé des contraintes
    ax.plot(x1, x2_temps, 'b-', linewidth=2.5, label='Contrainte temps (2x₁ + x₂ ≤ 200)', alpha=0.8)
    ax.plot(x1, x2_capacite, 'g-', linewidth=2.5, label='Contrainte capacité (x₁ + x₂ ≤ 150)', alpha=0.8)
    ax.axvline(x=100, color='orange', linewidth=2.5, label='Contrainte marché A (x₁ ≤ 100)', alpha=0.8)
    ax.axhline(y=100, color='purple', linewidth=2.5, label='Contrainte marché B (x₂ ≤ 100)', alpha=0.8)
    
    # Zone réalisable (calcul des sommets du polygone)
    vertices = np.array([
        [0, 0],
        [0, 100],
        [50, 100],
        [100, 0]
    ])
    
    polygon = Polygon(vertices, alpha=0.3, facecolor='lightblue', 
                     edgecolor='navy', linewidth=2, label='Zone réalisable')
    ax.add_patch(polygon)
    
    # Tracé des courbes iso-profit
    for benefice in [20000, 30000, max_benefice]:
        x2_iso = (benefice - 300*x1) / 200
        style = '--' if benefice != max_benefice else '-'
        width = 1.5 if benefice != max_benefice else 3
        alpha = 0.5 if benefice != max_benefice else 1
        label = f'Iso-profit {benefice:,.0f}€' if benefice != max_benefice else f'Iso-profit OPTIMAL {benefice:,.0f}€'
        ax.plot(x1, x2_iso, style, color='red', linewidth=width, alpha=alpha, label=label)
    
    # Point optimal
    ax.plot(x1_opt, x2_opt, 'r*', markersize=25, markeredgecolor='darkred', 
           markeredgewidth=2, label=f'Solution optimale', zorder=5)
    
    # Annotation du point optimal
    ax.annotate(f'Optimal\n({x1_opt:.0f}, {x2_opt:.0f})\n{max_benefice:,.0f}€',
               xy=(x1_opt, x2_opt), xytext=(x1_opt+15, x2_opt+15),
               fontsize=12, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.8', facecolor='yellow', alpha=0.8),
               arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.3', 
                             color='red', lw=2))
    
    # Configuration des axes
    ax.set_xlim(-5, 110)
    ax.set_ylim(-5, 110)
    ax.set_xlabel('Nombre de canapés modèle A (x₁)', fontsize=13, fontweight='bold')
    ax.set_ylabel('Nombre de canapés modèle B (x₂)', fontsize=13, fontweight='bold')
    ax.set_title('Optimisation de la production de canapés\nProgrammation Linéaire - Méthode Graphique', 
                fontsize=15, fontweight='bold', pad=20)
    
    # Grille et légende
    ax.grid(True, alpha=0.3, linestyle=':', linewidth=0.8)
    ax.legend(loc='upper right', fontsize=10, framealpha=0.95, shadow=True)
    
    # Ajout d'informations supplémentaires
    info_text = f"Production optimale : {x1_opt:.0f} × A + {x2_opt:.0f} × B = {max_benefice:,.0f}€\n"
    info_text += f"Temps utilisé : {2*x1_opt + x2_opt:.0f}h / 200h | Capacité : {x1_opt+x2_opt:.0f} / 150"
    ax.text(0.02, 0.98, info_text, transform=ax.transAxes, fontsize=11,
           verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# Génération du graphique
visualiser_solution(x1_opt, x2_opt, max_benefice)

---

## 🔬 Partie 4 : Analyse de sensibilité

Étudions comment la solution optimale varie en fonction des paramètres clés.

In [ ]:
def analyse_sensibilite_benefice():
    """Analyse de sensibilité sur les bénéfices unitaires"""
    
    # Variation du bénéfice du modèle A
    benefices_A = np.arange(200, 501, 20)
    resultats = []
    
    for b_A in benefices_A:
        c_temp = [-b_A, -200]
        result = so.linprog(c=c_temp, A_ub=probleme.A_ub, b_ub=probleme.b_ub, 
                          bounds=probleme.bounds, method='highs')
        if result.success:
            resultats.append({
                'Bénéfice_A': b_A,
                'x1_optimal': result.x[0],
                'x2_optimal': result.x[1],
                'Bénéfice_total': -result.fun
            })
    
    df = pd.DataFrame(resultats)
    
    # Visualisation
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Graphique 1 : Production optimale
    axes[0].plot(df['Bénéfice_A'], df['x1_optimal'], 'b-o', linewidth=2, markersize=6, label='Modèle A')
    axes[0].plot(df['Bénéfice_A'], df['x2_optimal'], 'g-s', linewidth=2, markersize=6, label='Modèle B')
    axes[0].axvline(x=300, color='red', linestyle='--', alpha=0.7, label='Valeur actuelle')
    axes[0].set_xlabel('Bénéfice unitaire modèle A (€)', fontweight='bold')
    axes[0].set_ylabel('Quantité optimale', fontweight='bold')
    axes[0].set_title('Impact du bénéfice A sur la production optimale', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Graphique 2 : Bénéfice total
    axes[1].plot(df['Bénéfice_A'], df['Bénéfice_total'], 'r-o', linewidth=2.5, markersize=6)
    axes[1].axvline(x=300, color='blue', linestyle='--', alpha=0.7, label='Valeur actuelle')
    axes[1].fill_between(df['Bénéfice_A'], df['Bénéfice_total'], alpha=0.3, color='red')
    axes[1].set_xlabel('Bénéfice unitaire modèle A (€)', fontweight='bold')
    axes[1].set_ylabel('Bénéfice total optimal (€)', fontweight='bold')
    axes[1].set_title('Impact du bénéfice A sur le bénéfice total', fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return df

# Exécution de l'analyse
df_sensibilite = analyse_sensibilite_benefice()

In [ ]:
# Affichage du tableau de sensibilité
print("\n📊 Tableau de sensibilité (extrait)")
print("=" * 70)
display(df_sensibilite[::5].style.format({
    'Bénéfice_A': '{:.0f} €',
    'x1_optimal': '{:.0f}',
    'x2_optimal': '{:.0f}',
    'Bénéfice_total': '{:,.0f} €'
}).background_gradient(subset=['Bénéfice_total'], cmap='Greens'))

---

## 📦 Partie 5 : Extension - Contraintes de matières premières

L'entreprise dispose également de ressources limitées en matières premières :

| Matière | Modèle A | Modèle B | Stock disponible |
|---------|----------|----------|------------------|
| Bois | 2 unités | 1 unité | 125 unités |
| Visserie | 0.2 kg | 0.2 kg | 55 kg |
| Aluminium | 1 m | 1 m | 300 m |
| Acier | 2 kg | 2 kg | 100 kg |
| Caoutchouc | 1 m² | 1 m² | 60 m² |
| Mousse | 1 m³ | 1 m³ | 100 m³ |
| Tissu | 2 m | 2 m | 220 m |

In [ ]:
# Problème étendu avec matières premières
class ProblemeEtendu:
    """Problème d'optimisation avec contraintes de matières premières"""
    
    def __init__(self):
        self.c = [-300, -200]
        
        # Matrice des contraintes étendue
        self.A_ub = np.array([
            [2, 1],      # Temps
            [1, 1],      # Capacité totale
            [2, 1],      # Bois
            [0.2, 0.2],  # Visserie
            [1, 1],      # Aluminium
            [2, 2],      # Acier
            [1, 1],      # Caoutchouc
            [1, 1],      # Mousse
            [2, 2]       # Tissu
        ])
        
        self.b_ub = np.array([200, 150, 125, 55, 300, 100, 60, 100, 220])
        self.bounds = [(0, 100), (0, 100)]
        
        self.noms_contraintes = [
            'Temps', 'Capacité', 'Bois', 'Visserie', 
            'Aluminium', 'Acier', 'Caoutchouc', 'Mousse', 'Tissu'
        ]

probleme_etendu = ProblemeEtendu()

# Résolution
result_etendu = so.linprog(
    c=probleme_etendu.c,
    A_ub=probleme_etendu.A_ub,
    b_ub=probleme_etendu.b_ub,
    bounds=probleme_etendu.bounds,
    method='highs'
)

if result_etendu.success:
    x1_ext, x2_ext = result_etendu.x
    benefice_ext = -result_etendu.fun
    
    print("\n" + "=" * 70)
    print("🏭 SOLUTION AVEC CONTRAINTES DE MATIÈRES PREMIÈRES")
    print("=" * 70)
    print(f"\n📦 Production optimale :")
    print(f"   • Canapés modèle A : {x1_ext:.0f} unités")
    print(f"   • Canapés modèle B : {x2_ext:.0f} unités")
    print(f"\n💰 Bénéfice maximal : {benefice_ext:,.0f} €")
    print(f"\n📉 Comparaison avec le cas sans contraintes de matières :")
    print(f"   • Perte de production : {(x1_opt + x2_opt) - (x1_ext + x2_ext):.0f} unités")
    print(f"   • Perte de bénéfice : {max_benefice - benefice_ext:,.0f} € ({100*(max_benefice - benefice_ext)/max_benefice:.1f}%)")
    
    # Analyse des contraintes saturées
    print(f"\n🔍 Contraintes saturées (limitantes) :")
    utilisation = probleme_etendu.A_ub @ result_etendu.x
    for i, (nom, util, dispo) in enumerate(zip(probleme_etendu.noms_contraintes, utilisation, probleme_etendu.b_ub)):
        taux = 100 * util / dispo
        if taux > 99:
            print(f"   ⚠️  {nom:12s} : {util:.1f} / {dispo:.0f} ({taux:.1f}%) - SATURÉE")
        elif taux > 90:
            print(f"   ⚡ {nom:12s} : {util:.1f} / {dispo:.0f} ({taux:.1f}%) - Presque saturée")
    
    print("=" * 70)
else:
    print("❌ Problème infaisable avec les contraintes de matières premières")

---

## 📊 Partie 6 : Tableau de bord récapitulatif

In [ ]:
# Création d'un tableau comparatif
comparaison = pd.DataFrame({
    'Scénario': ['Sans contraintes MP', 'Avec contraintes MP', 'Écart'],
    'Canapés A': [x1_opt, x1_ext, x1_opt - x1_ext],
    'Canapés B': [x2_opt, x2_ext, x2_opt - x2_ext],
    'Total canapés': [x1_opt + x2_opt, x1_ext + x2_ext, (x1_opt + x2_opt) - (x1_ext + x2_ext)],
    'Bénéfice (€)': [max_benefice, benefice_ext, max_benefice - benefice_ext]
})

print("\n📈 TABLEAU RÉCAPITULATIF")
print("=" * 80)
display(comparaison.style.format({
    'Canapés A': '{:.0f}',
    'Canapés B': '{:.0f}',
    'Total canapés': '{:.0f}',
    'Bénéfice (€)': '{:,.0f}'
}).background_gradient(subset=['Bénéfice (€)'], cmap='RdYlGn'))

---

## 🎓 Conclusions et recommandations

### 📌 Résultats clés

1. **Sans contraintes de matières premières** :
   - Production optimale : **50 canapés A + 100 canapés B**
   - Bénéfice maximal : **35 000 €**
   - Contrainte limitante : **Capacité marché B**

2. **Avec contraintes de matières premières** :
   - La solution est plus contrainte
   - Perte de bénéfice due aux limitations de matières
   - Identification des goulots d'étranglement

### 💡 Recommandations stratégiques

1. **Court terme** :
   - Privilégier le modèle B (meilleur ratio bénéfice/temps)
   - Négocier avec fournisseurs pour les matières saturées
   
2. **Moyen terme** :
   - Investir dans l'augmentation des capacités de production
   - Étudier la possibilité d'augmenter les prix du modèle A
   
3. **Long terme** :
   - Diversifier la gamme de produits
   - Optimiser la chaîne logistique

### 🔧 Limites et extensions possibles

- **Hypothèses simplificatrices** : Production continue, pas de coûts fixes
- **Extensions possibles** :
  - Modèle stochastique avec incertitude sur la demande
  - Optimisation multi-périodes
  - Programmation en nombres entiers
  - Analyse coût-bénéfice des investissements

---

## 📚 Références et ressources

### Bibliothèques utilisées
- **NumPy** : Calcul numérique et algèbre linéaire
- **SciPy** : Optimisation (algorithme HiGHS)
- **Matplotlib** : Visualisation de données
- **Pandas** : Manipulation et analyse de données

### Méthodes
- **Programmation linéaire** : Méthode du simplexe
- **Analyse de sensibilité** : Étude paramétrique
- **Visualisation** : Méthode graphique

---

*Notebook réalisé par Emmanuel Paguiel BOUENDO - Master MÉcEn 2024-2025*